In [30]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

In [2]:
import IPython
IPython.utils.encoding.getdefaultencoding()

'UTF-8'

In [3]:
import sys
sys.getdefaultencoding()

'utf-8'

In [4]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

In [5]:
# read files
def readInputFiles(train_file_path, test_file_path):
    train = pd.read_csv(train_file_path)
    test = pd.read_csv(test_file_path)
    train = train.sample(frac=1)
    return train, test
    
train, test = readInputFiles('./dataset/train_new.csv', './dataset/test_new.csv')

In [6]:
EMBEDDING_FILE = './embeddings/fasttext/crawl-300d-2M.vec'

In [21]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

def preprocess_data():
    max_features = 20000
    maxlen = 200
    
    y = train[list_classes].values
    y_test = test[list_classes].values
    list_sentences_train = train["comment_text"].fillna("CVxTz").values
    list_sentences_test = test["comment_text"].fillna("CVxTz").values
   
    tokenizer = Tokenizer(num_words=max_features)

    tokenizer.fit_on_texts(list(list_sentences_train))
    
    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
    
    X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
    
    return max_features, maxlen, X_train, X_test, y, y_test, tokenizer

max_features, maxlen, X_train, X_test, y, y_test, tokenizer = preprocess_data()

In [8]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

In [9]:
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf-8'))

In [10]:
embed_size = 300

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [11]:
def get_model():
    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [12]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [13]:
def createModel():
    model = get_model()
    batch_size = 32
    epochs=2
    model.fit(X_train, y, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    return model

## Fit model if doesn't exist already 

In [14]:
print("start fitting...")
file_path = "pooledgru_fasttext_model_v2.h5"
if os.path.isfile(file_path):
    print ("Model already exists. Loading from path ", file_path)
    model = load_model(file_path)
else:
    print ("Model doesn't exist already, training model and saving at path ", file_path)
    model = createModel()
    model.save(file_path)

start fitting...
Model already exists. Loading from path  pooledgru_fasttext_model_v2.h5


In [15]:
print("start predicting...")
y_pred = model.predict(X_test, batch_size=1024)
print ("done")

start predicting...
done


In [16]:
submission = pd.DataFrame.from_dict({'id': test['id']})
for idx, col in enumerate(list_classes):
    submission[col] = y_pred[:,idx]
submission.to_csv('submission_pooled.csv', index=False)

## Calculate AUC loss

In [17]:
from score import calc_auc_score, calc_log_loss

In [27]:
def get_scores(test, preds, fallback_preds_filename):
    try: 
        true = test
    except NameError:
        true = pd.read_csv('../dataset/test_new.csv')
    try: 
        y_pred = preds
    except NameError:
        pred = pd.read_csv(fallback_preds_filename)
        y_pred = pred[list_classes].values

    y_true = true[list_classes].values

    loss = calc_log_loss(y_true, y_pred)
    auc = calc_auc_score(y_true, y_pred)
    return loss, auc

In [29]:
true = pd.read_csv('./dataset/test_new.csv')
pred = y_pred

loss_, aucs = get_scores(true, pred, fallback_preds_filename=None)

In [31]:
print ("Log loss = ", loss_)
print ("AUC Score = ", aucs)

Log loss =  0.041929289638716
AUC Score =  0.9878130862611308


## Demo

In [26]:
zgg = tokenizer.texts_to_sequences(["They're not stupid"])
aaa = sequence.pad_sequences(zgg, maxlen=200)
np.set_printoptions(suppress=True)
model.predict(aaa)

array([[0.9171239 , 0.00300967, 0.20183349, 0.00022819, 0.4994991 ,
        0.00364819]], dtype=float32)